In [1]:
!pip install --upgrade transformers huggingface_hub


In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message="`resume_download` is deprecated")


In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

# Last inn datasettet
data_path = 'renset_fil_1.csv'  
data = pd.read_csv(data_path, sep=';')

# Forbehandling av datasettet
data['Spørsmål_uten_stopwords'].fillna('manglende', inplace=True)
data['combined_info'] = data[['Svar', 'Les mer her…', 'Link']].apply(lambda x: f"{x['Svar']} {x['Les mer her…']} {x['Link']}", axis=1)

# Initialiser modell og tokenizer
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Funksjon for å generere embeddings
def encode(texts):
    with torch.no_grad():
        return model(**tokenizer(texts, return_tensors='pt', padding=True, truncation=True)).last_hidden_state.mean(dim=1)


question_embeddings = encode(data['Spørsmål_uten_stopwords'].tolist())

def find_most_similar_question(user_question, k=1):
    user_question_embedding = encode([user_question])
    cos_sim = torch.nn.functional.cosine_similarity(user_question_embedding, question_embeddings)
    top_results = torch.topk(cos_sim, k)

    # Sjekk og hent det faktiske indeksnummeret fra tensorobjektet
    best_index = top_results.indices[0].item()  # Bruk .item() for å konvertere tensor til et Python heltall

    return data.iloc[best_index]['combined_info']

# Lagre modellen og tokenizeren
model_path = './saved_model_L12'
tokenizer_path = './saved_tokenizer_L12'
model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)


('./saved_tokenizer_L12\\tokenizer_config.json',
 './saved_tokenizer_L12\\special_tokens_map.json',
 './saved_tokenizer_L12\\unigram.json',
 './saved_tokenizer_L12\\added_tokens.json',
 './saved_tokenizer_L12\\tokenizer.json')

In [4]:
# Laste inn modellen og tokenizeren på nytt (i et nytt script eller en ny notebook)
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)



In [5]:
# Eksempel på bruk
user_question = "hvor lenge jobber jeg i husbanken"
print(find_most_similar_question(user_question))


Normalarbeidstiden i Husbanken varierer avhengig av sesongen:Fra 15. mai til 14. september er arbeidstiden kl. 08.00-15.00. Fra 15. september til 14. mai er arbeidstiden kl. 08.00-15.45 Les mer her… https://husnettet-d/HR-Portalen/For-ansatte/Sider/Arbeidstid


In [6]:
import gradio as gr
import re

def format_links_in_text(text):
    # Finn alle URL i teksten og erstatt dem med HTML-lenker
    url_pattern = r'https?://\S+/([^/]+)$'  # Regex for å fange opp den siste delen av URL-en (navnet på filen)
    text_with_links = re.sub(url_pattern, r"<a href='\g<0>' target='_blank'>\g<1></a>", text)
    return text_with_links

def answer_question(user_question):
    response = find_most_similar_question(user_question)
    response_with_links = format_links_in_text(response)
    return response_with_links

interface = gr.Interface(fn=answer_question,
                         inputs=gr.Textbox(lines=2, placeholder="Skriv inn ditt spørsmål her..."),
                         outputs="html",  # Endre til HTML-utgang for å tillate HTML-format
                         title="Chatbot",
                         description="Skriv inn et spørsmål for å få et svar fra chatboten.")

if __name__ == "__main__":
    interface.launch()


Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.
